In [1]:
import tensorflow as tf

#multi-gpu tutorial

#https://www.tensorflow.org/guide/distributed_training?hl=ko

#각각의 GPU마다 복제본 생성, 장치간 통신에는 NCCL 사용
mirrored_strategy = tf.distribute.MirroredStrategy()
#일부만 사용하고 싶을 경우
#mirrored_strategy = tf.distributed.MirroredStrategy(devices=["/gpu:0","/gpu:1"])
#통신방법 바꾸고싶은 경우, default = NcclAllReduce, ReductionToOneDevice
#mirrored_strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2022-02-07 19:36:51.417940: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-07 19:36:52.722550: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22345 MB memory:  -> device: 0, name: Graphics Device, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-02-07 19:36:52.723689: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22345 MB memory:  -> device: 1, name: Graphics Device, pci bus id: 0000:65:00.0, compute capability: 8.6


In [2]:
print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))

Number of devices: 2


In [3]:
#mnist 예제

import numpy as np
import os

fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Adding a dimension to the array -> new shape == (28, 28, 1)
# We are doing this because the first layer in our model is a convolutional
# layer and it requires a 4D input (batch_size, height, width, channels).
# batch_size dimension will be added later on.
train_images = train_images[..., None]
test_images = test_images[..., None]

# Getting the images in [0, 1] range.
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

In [4]:
# 입력 파이프라인 설정
BUFFER_SIZE = len(train_images)

BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync

EPOCHS = 10

In [5]:
#dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE) 
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE) 

train_dist_dataset = mirrored_strategy.experimental_distribute_dataset(train_dataset)
test_dist_dataset = mirrored_strategy.experimental_distribute_dataset(test_dataset)

2022-02-07 19:36:55.543808: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_UINT8
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 60000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 28
        }
        dim {
          size: 28
        }
        dim {
          size: 1
        }
      }
      shape {
      }
    }
  }
}

2022-02-07 19:36:55.580499: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy 

In [6]:
#model 생성
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])

    return model

In [7]:
# Create a checkpoint directory to store the checkpoints.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

In [8]:
with mirrored_strategy.scope():
  #scope 안에서 분산 모델 정의
  # Set reduction to `none` so we can do the reduction afterwards and divide by
  # global batch size.
  #sparsecategoricalcrossentropy 훈련데이터의 label이 정수일 경우 
  #label을 주면 내부적으로 one-hot vector로 변환해 로스 계산
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True,
      reduction=tf.keras.losses.Reduction.NONE)
  #분산에서는 reduction.none으로 설정하고 직접 명시하기
  #왜냐하면 글로벌 배치 고려해야하기 때문
  def compute_loss(labels, predictions):
    per_example_loss = loss_object(labels, predictions)
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

In [9]:
with mirrored_strategy.scope():
  test_loss = tf.keras.metrics.Mean(name='test_loss')

  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='test_accuracy')

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [10]:
# model, optimizer, and checkpoint must be created under `strategy.scope`.
with mirrored_strategy.scope():
  model = create_model()

  optimizer = tf.keras.optimizers.Adam()

  checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

In [11]:
checkpoint

In [12]:
def train_step(inputs):
    images, labels = inputs

    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = compute_loss(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_accuracy.update_state(labels, predictions)
    return loss 
#여기서 각각 GPU에서 loss를 계산한 후 global reduce를 하는데, od에서는
#1. boudning box 계산을 전부 global에서 하거나
#2. 경계면만 glbal 에서 처리

def test_step(inputs):
    images, labels = inputs

    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss.update_state(t_loss)
    test_accuracy.update_state(labels, predictions)

In [13]:
# `run` replicates the provided computation and runs it
# with the distributed input.
@tf.function
def distributed_train_step(dataset_inputs):
  print(dataset_inputs)
  per_replica_losses = mirrored_strategy.run(train_step, args=(dataset_inputs,))
  return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

@tf.function
def distributed_test_step(dataset_inputs):
  return mirrored_strategy.run(test_step, args=(dataset_inputs,))

for epoch in range(EPOCHS):
  # TRAIN LOOP
  total_loss = 0.0
  num_batches = 0
  for x in train_dist_dataset:
    total_loss += distributed_train_step(x)
    num_batches += 1
  train_loss = total_loss / num_batches

  # TEST LOOP
  for x in test_dist_dataset:
    distributed_test_step(x)

  if epoch % 2 == 0:
    checkpoint.save(checkpoint_prefix)

  template = ("Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, "
              "Test Accuracy: {}")
  print (template.format(epoch+1, train_loss,
                         train_accuracy.result()*100, test_loss.result(),
                         test_accuracy.result()*100))

  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

(PerReplica:{
  0: <tf.Tensor 'dataset_inputs:0' shape=(64, 28, 28, 1) dtype=float32>,
  1: <tf.Tensor 'dataset_inputs_1:0' shape=(64, 28, 28, 1) dtype=float32>
}, PerReplica:{
  0: <tf.Tensor 'dataset_inputs_2:0' shape=(64,) dtype=uint8>,
  1: <tf.Tensor 'dataset_inputs_3:0' shape=(64,) dtype=uint8>
})
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1


/home/gmpark/anaconda3/envs/ed/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


(PerReplica:{
  0: <tf.Tensor 'dataset_inputs:0' shape=(64, 28, 28, 1) dtype=float32>,
  1: <tf.Tensor 'dataset_inputs_1:0' shape=(64, 28, 28, 1) dtype=float32>
}, PerReplica:{
  0: <tf.Tensor 'dataset_inputs_2:0' shape=(64,) dtype=uint8>,
  1: <tf.Tensor 'dataset_inputs_3:0' shape=(64,) dtype=uint8>
})
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1


2022-02-07 19:38:10.467654: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
2022-02-07 19:38:12.228496: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-07 19:38:12.309665: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
2022-02-07 19:38:14.840598: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(PerReplica:{
  0: <tf.Tensor 'dataset_inputs:0' shape=(48, 28, 28, 1) dtype=float32>,
  1: <tf.Tensor 'dataset_inputs_1:0' shape=(48, 28, 28, 1) dtype=float32>
}, PerReplica:{
  0: <tf.Tensor 'dataset_inputs_2:0' shape=(48,) dtype=uint8>,
  1: <tf.Tensor 'dataset_inputs_3:0' shape=(48,) dtype=uint8>
})
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
Epoch 1, Loss: 0.5852965116500854, Accuracy: 79.00833129882812, Test Loss: 0.42411914467811584, Test Accuracy: 85.18000030517578
Epoch 2, Loss: 0.3680661618709564, Accuracy: 86.89167022705078, Test Loss: 0.35276558995246887, Test Accuracy: 87.84000396728516
Epoch 3, Loss: 0.32106900215148926, Accuracy: 88.48999786376953, Test Loss: 0.34344345331192017, Test Accuracy: 87.7300033569336
Epoch 4, Loss: 0.28980493545532227, Accuracy: 89.61166381835938, Test Loss: 0.3161250054836273, Test Accuracy: 88.95999908447266
Epoch 5, Loss: 0.267914742231369, Accuracy: 90.19833374023438, Test Loss: 0.29504385590553284,

In [ ]:
#evaluation
eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='eval_accuracy')

new_model = create_model()
new_optimizer = tf.keras.optimizers.Adam()

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE)

In [ ]:
@tf.function
def eval_step(images, labels):
  predictions = new_model(images, training=False)
  eval_accuracy(labels, predictions)l

In [ ]:
checkpoint = tf.train.Checkpoint(optimizer=new_optimizer, model=new_model)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

for images, labels in test_dataset:
  eval_step(images, labels)

print ('전략을 사용하지 않고, 저장된 모델을 복원한 후의 정확도: {}'.format(
    eval_accuracy.result()*100))